In [ ]:
# replace the standard sqlite3 module with pysqlite3
# for compatibility with Chroma
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


import langchain
import os
import bs4
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# load environment variables
load_dotenv()

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# credential json not required if you are working within vertex AI
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/workspaces/LLM-agent-with-Gemini/fleet-anagram-244304-7dafcc771b2f.json"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # only if you are using text embedding model from google

In [ ]:
search = TavilySearchResults(max_results=3)
result = search.invoke("What is Julia language known for?")

# Formatting
for i, item in enumerate(result, start=1):
    print(f"{i}. URL: {item['url']}\nContent: {item['content']}\n")

In [ ]:
web_paths = [
    "https://google.github.io/styleguide/pyguide.html",
    "https://google.github.io/styleguide/Rguide.html",
    # "https://google.github.io/styleguide/cppguide.html",
    # "https://google.github.io/styleguide/go/",
    # "https://google.github.io/styleguide/javaguide.html",
    # "https://google.github.io/styleguide/jsguide.html",
    # "https://google.github.io/styleguide/tsguide.html"
]

# Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

docs = []
for path in web_paths:
    loader = WebBaseLoader(web_paths=(path,),
                        #    bs_kwargs={"parse_only": bs4_strainer}
    )
    docs += loader.load()

In [ ]:
len(docs[0].page_content)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
all_splits[-1].metadata

In [ ]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "programming_style_guide_search",
    "Reference for information about styling rules for programming languages",
)

In [ ]:
tools = [search, retriever_tool]

In [ ]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

In [ ]:
import json

def print_formatted_response(response):
    for i in range(4):
        if i == 0:
            print(f"Question: {response['messages'][0].content}")
        elif i == 1:
            function_call = response['messages'][i].additional_kwargs.get('function_call', {})
            name = function_call.get('name', '')
            print(f"Tool: {name}")
        elif i == 2:
            data = json.loads(response['messages'][2].content)
            print("Citation:")
            for item in data:
                print(item['url'])
        else:
            print(f"Answer: {response['messages'][3].content}")
        print()

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What are the differences between langchain and llamaindex, search if it's not in retriever")]}
)
response["messages"]

In [ ]:
print(response["messages"][-1].content)

In [ ]:
# streamin
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is Vajra")]}
):
    print(chunk)
    print("----")

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi, I'm Hanyu")]}, config
):
    print(chunk)
    print()

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's my name?")]}, config
):
    print(chunk)
    print()

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Suggest some use cases for LLM Agent")]}, config
)

print(response["messages"][-1].content)
    